In [1]:
import pandas as pd
import time
import warnings

warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
%%time
data = pd.read_csv("GKX_20201231.csv")
data.head()

CPU times: user 1min 22s, sys: 36 s, total: 1min 57s
Wall time: 2min 40s


,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
0,10006,19260130,65400.0,0.032732,110.2500,600,NaN,NaN,NaN,NaN,...,0.006857,NaN,NaN,NaN,NaN,NaN,0.000066,NaN,NaN,NaN
1,10022,19260130,11200.0,0.017857,57.0000,200,NaN,NaN,NaN,NaN,...,0.018018,NaN,NaN,NaN,NaN,NaN,0.000003,NaN,NaN,NaN
2,10030,19260130,23400.0,0.161667,174.2500,156,NaN,NaN,NaN,NaN,...,0.004158,NaN,NaN,NaN,NaN,NaN,0.000023,NaN,NaN,NaN
3,10049,19260130,18500.0,0.141892,83.5000,250,NaN,NaN,NaN,NaN,...,0.006780,NaN,NaN,NaN,NaN,NaN,0.000022,NaN,NaN,NaN
4,10057,19260130,6125.0,-0.035714,11.8125,500,NaN,NaN,NaN,NaN,...,0.071795,NaN,NaN,NaN,NaN,NaN,0.000010,NaN,NaN,NaN


In [3]:
start_date = 19570101
end_date = 20161231

data1 = data[(data['DATE'] >= start_date) & (data['DATE'] <= end_date)]
data1['DATE'] = pd.to_datetime(data1['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
data1.head()

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/2929358852.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['DATE'] = pd.to_datetime(data1['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)


,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
306536,10006,1957-01-31,82249.000,0.064378,62.000,1412,1.122846,1.260784,0.047180,9.569953,...,0.013234,9.411565e-08,0.015453,0.008058,0.355638,0.460420,1.120996e-07,37.0,NaN,NaN
306537,10014,1957-01-31,3903.375,0.095238,2.875,1487,0.426734,0.182102,-0.275641,6.237836,...,0.033305,6.610609e-06,0.047619,0.033495,1.152126,1.169610,9.229146e-08,NaN,NaN,NaN
306538,10022,1957-01-31,9273.250,0.102041,6.750,1514,1.066449,1.137313,-0.025490,7.008844,...,0.016023,2.286832e-06,0.020833,0.015589,0.815777,0.679803,1.181757e-07,NaN,NaN,NaN
306539,10030,1957-01-31,54465.875,-0.047091,43.000,1207,0.926038,0.857547,0.018171,9.825337,...,0.015295,1.464273e-07,0.039326,0.015849,0.739302,1.333656,6.126699e-08,NaN,NaN,NaN
306540,10057,1957-01-31,40250.000,-0.090062,73.250,500,1.247748,1.556875,0.025785,7.901007,...,0.005954,1.380375e-06,0.056856,0.019945,0.755510,0.410391,3.315790e+00,NaN,NaN,NaN


In [4]:
# now dealing with missing values
# the authors replace missing values with the cross section median
data1.isnull().sum()

permno            0
DATE              0
mvel1          3040
RET               0
prc           19073
              ...  
std_turn     305623
zerotrade    309735
sic2         260072
bm           994113
bm_ia        994113
Length: 101, dtype: int64

In [5]:
# now filling the missing values with the cross sectional median
attributes = list(set(data1.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))
for col in attributes:
    data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=ke

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=ke

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=ke

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=ke

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/

...

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/lib/python3.9/site-packages/

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/3179544859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.median()))


In [6]:
# lets see the filling of missing values w/ the cross sectional median
data1.isnull().sum()

permno            0
DATE              0
mvel1             0
RET               0
prc           19073
              ...  
std_turn          0
zerotrade         0
sic2         260072
bm            72077
bm_ia         72077
Length: 101, dtype: int64

In [7]:
attributes = list(set(data1.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))
for col in attributes:
    data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.fillna(0)))

/var/folders/w2/fhp_35t568z39d8xb73mfj880000gp/T/ipykernel_3208/2486486636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col] = data1.groupby('DATE')[col].transform(lambda x: x.fillna(x.fillna(0)))


In [8]:
data1.isnull().sum()

permno            0
DATE              0
mvel1             0
RET               0
prc           19073
              ...  
std_turn          0
zerotrade         0
sic2         260072
bm                0
bm_ia             0
Length: 101, dtype: int64

In [9]:
# now we need to transform the SIC Codes to dummies to be able to extract insights from them
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(data_ch['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    data_ch_d = pd.concat([data_ch,sic_dummies],axis=1)
    data_ch_d.drop(['prc','SHROUT','mve0','sic2'],inplace=True,axis=1) # dropping cols not needed
    return data_ch_d

In [10]:
data_d = get_sic_dummies(data1)

In [11]:
data_d.head()

,permno,DATE,mvel1,RET,beta,betasq,chmom,dolvol,idiovol,indmom,...,sic_79,sic_80,sic_81,sic_82,sic_83,sic_84,sic_86,sic_87,sic_89,sic_99
306536,10006,1957-01-31,82249.000,0.064378,1.122846,1.260784,0.047180,9.569953,0.025742,0.046197,...,False,False,False,False,False,False,False,False,False,False
306537,10014,1957-01-31,3903.375,0.095238,0.426734,0.182102,-0.275641,6.237836,0.069611,0.046197,...,False,False,False,False,False,False,False,False,False,False
306538,10022,1957-01-31,9273.250,0.102041,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046197,...,False,False,False,False,False,False,False,False,False,False
306539,10030,1957-01-31,54465.875,-0.047091,0.926038,0.857547,0.018171,9.825337,0.021700,0.046197,...,False,False,False,False,False,False,False,False,False,False
306540,10057,1957-01-31,40250.000,-0.090062,1.247748,1.556875,0.025785,7.901007,0.025506,0.046197,...,False,False,False,False,False,False,False,False,False,False


We also need additional data source for our macroeconomic influencers, like risk free rates etc. The data we source is from Professor Goyal's Website (as suggested by Yanlin Bao)

In [12]:
macro_data = pd.read_csv("PredictorData2023.xlsx - Monthly.csv")
macro_data = macro_data[(macro_data['yyyymm'] >= start_date//100) & (macro_data['yyyymm'] <= end_date//100)].reset_index(drop = True)
## constructing our predictors
ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']
macro_data['Index'] = macro_data['Index'].str.replace(',','').astype('float64')
macro_data['dp_sp'] = macro_data['D12']/macro_data['Index']
macro_data['ep_sp'] = macro_data['E12']/macro_data['Index']
macro_data.rename({'b/m':'bm_sp'},axis=1,inplace=True)
macro_data['tms'] = macro_data['lty']-macro_data['tbl']
macro_data['dfy'] = macro_data['BAA']-macro_data['AAA']
macro_data = macro_data[['yyyymm']+ma_predictors]
macro_data['yyyymm'] = pd.to_datetime(macro_data['yyyymm'],format='%Y%m')+pd.offsets.MonthEnd(0)
macro_data.head()

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar
0,1957-01-31,0.038835,0.076178,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902
1,1957-02-28,0.040067,0.078671,0.584994,0.030173,0.0310,0.0018,0.0080,0.001056
2,1957-03-31,0.039220,0.077080,0.599819,0.026600,0.0308,0.0023,0.0077,0.000330
3,1957-04-30,0.037822,0.074480,0.576098,0.027421,0.0307,0.0038,0.0077,0.000302
4,1957-05-31,0.036475,0.071965,0.564039,0.028849,0.0306,0.0042,0.0078,0.000482


In [13]:
## now lets construct a complete dataset whilst also normalising our features via Min-Max-Scaling
from sklearn.preprocessing import MinMaxScaler
def interactions(data1, macro_data, attributes, ma_predictors, minmax=True):
    # construct interactions between firm characteristics and macroeconomic predictors; analog to Yanlin Bao
    data = data1.copy()
    data_ma_long = pd.merge(data[['DATE']],macro_data,left_on='DATE',right_on='yyyymm',how='left')
    data = data.reset_index(drop=True)
    data_ma_long = data_ma_long.reset_index(drop=True)
    for fc in attributes:
        for mp in ma_predictors:
            data[fc+'*'+mp] = data[fc]*data_ma_long[mp]

    features = list(set(data.columns).difference({'permno','DATE','RET'})) # a list storing all 920 features used
    if minmax:
        X = MinMaxScaler((-1,1)).fit_transform(data[features])
        X = pd.DataFrame(X, columns=features)
    else:
        X = data[features]
    y = data['RET']
    print(f"shape of our data: {data.shape}")
    return X, y

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
del(data, data1)

In [15]:
import numpy as np

now we need to split the dataset into training, validation and testing sets
The authors use 1957-1974 for training, 1987-2016 for out-of-sample testing and the 12 years in between for tuning hyperparameters

In [ ]:
## so lets split our dataset into the different subsamples
stdt_vld = np.datetime64('1975-01-31')
stdt_tst = np.datetime64('1987-01-31')

In [ ]:
# now defining the split function for our test_sample; afterwards just recompose for validation & training, then i saved the files as .pkl for further work
def tst(data):
        X_tst, y_tst = interactions(data[data['DATE']>=stdt_tst],macro_data[macro_data['yyyymm']>=stdt_tst],attributes,ma_predictors)
        return X_tst, y_tst